In [45]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

test_df =pd.read_csv("data/test.csv")
train_df=pd.read_csv("data/train.csv")

train_df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [46]:
#analyse der daten 
data_info = train_df.dtypes
missing_values = train_df.isnull().sum()
analysis_table = pd.DataFrame({
    'Data Type' : data_info,
    'Missing Values' : missing_values
})
 


test_info = test_df.dtypes
missing_values = test_df.isnull().sum()
analysis_table_test = pd.DataFrame({
    
    'Data Type' : data_info,
    'Missing Values' : missing_values
})


analysis_table_test

,Data Type,Missing Values
Age,float64,86.0
Cabin,object,327.0
Embarked,object,0.0
Fare,float64,1.0
Name,object,0.0
Parch,int64,0.0
PassengerId,int64,0.0
Pclass,int64,0.0
Sex,object,0.0
SibSp,int64,0.0


In [47]:
# Fehlende Werte für 'Age' und 'embarked' mit dem Durchschnitt und S  ersetzen
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
train_df['Embarked'] = train_df['Embarked'].fillna('S')
median_test=test_df['Age'].median()
median_train=train_df['Age'].median()
if median_train < median_test :
    test_df['Age'] = test_df['Age'].fillna(median_train)
else :
    test_df['Age'] = test_df['Age'].fillna(median_test)
median_fare = train_df['Fare'].median()
test_df['Fare'] = test_df['Fare'].fillna(median_fare)


test_df['Age']


0      34.5
1      47.0
2      62.0
3      27.0
4      22.0
       ... 
413    27.0
414    39.0
415    38.5
416    27.0
417    27.0
Name: Age, Length: 418, dtype: float64

In [48]:


analysis_table_test

,Data Type,Missing Values
Age,float64,86.0
Cabin,object,327.0
Embarked,object,0.0
Fare,float64,1.0
Name,object,0.0
Parch,int64,0.0
PassengerId,int64,0.0
Pclass,int64,0.0
Sex,object,0.0
SibSp,int64,0.0


In [49]:
# Kategorische Spalten in numerische umwandeln male =1 woman =0  C=0  Q=1 S=
le = LabelEncoder()
for dataset in [train_df, test_df]:
    dataset['Sex'] = le.fit_transform(dataset['Sex'])
    dataset['Embarked'] = le.fit_transform(dataset['Embarked'])
    

test_df ['Sex']
    


0      1
1      0
2      1
3      1
4      0
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 418, dtype: int64

In [50]:
train_df['Embarked']

0      2
1      0
2      2
3      2
4      2
      ..
886    2
887    2
888    2
889    0
890    1
Name: Embarked, Length: 891, dtype: int64

In [51]:
# Wichtige Merkmale definieren
features = [ 'Sex', 'Age', 'SibSp',  'Embarked']
X_train = train_df[features]
y_train = train_df['Survived']
X_test = test_df[features]
X_train

,Sex,Age,SibSp,Embarked
0,1,22.0,1,2
1,0,38.0,1,0
2,0,26.0,0,2
3,0,35.0,1,2
4,1,35.0,0,2
...,...,...,...,...
886,1,27.0,0,2
887,0,19.0,0,2
888,0,28.0,1,2
889,1,26.0,0,0


In [52]:
# daten aufteilen und training dürchführen 20 %  der daten für die die validierung   und wird maximum 200 mal durchlaufen 
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=400)
model.fit(X_tr, y_tr)

LogisticRegression(max_iter=400)

In [53]:
# trainiertes Modell wird genutzt, um Vorhersagen für die Daten in X_val zu machen.
y_pred = model.predict(X_val)




In [54]:
# test 
# Vorhersagen für das Testset
test_predictions = model.predict(X_test)

# Speichern der Einreichungen
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': test_predictions})
output.to_csv('submission.csv', index=False)
print("Einreichungsdatei erstellt.")

Einreichungsdatei erstellt.
